## Data preperation for Citibike Tableau project - Jersey City June - July 2022 & 2023

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

### Combining files to be used 
#### June-July 2022 and June-July 2023 Jersey City data has been selected for this analysis. 

In [3]:
# List of NYC citibike data file names to be combined 
file_list = ['Resources/JC-202206-citibike-tripdata.csv', 'Resources/JC-202207-citbike-tripdata.csv',
             'Resources/JC-202306-citibike-tripdata.csv', 'Resources/JC-202307-citibike-tripdata.csv']

# Create an empty list to store each dataframe
trip_list = []

# Loop through each file in the list
for file in file_list:
    # Read the file into a dataframe
    # https://www.statology.org/pandas-read-csv-dtype/
    trip_df = pd.read_csv(file, dtype = {'start_station_id': str,
                                        'end_station_id': str} )

    # Append the dataframe to the list
    trip_list.append(trip_df)

# Concatenate all dataframes in the list into one dataframe
trip_df = pd.concat(trip_list)

# Show dataframe
trip_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,EEA3CD3B084370F4,classic_bike,2022-06-30 19:17:16,2022-06-30 19:27:13,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,40.737430,-74.035710,40.754090,-74.031600,casual
1,BF568C9B3C5FAE94,classic_bike,2022-06-02 12:46:44,2022-06-02 15:08:33,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,40.737430,-74.035710,40.754090,-74.031600,casual
2,DF6B06D460481A7E,classic_bike,2022-06-05 14:17:34,2022-06-05 14:27:12,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,40.737430,-74.035710,40.754090,-74.031600,casual
3,A26CDB92CDF7E0EE,electric_bike,2022-06-01 18:57:12,2022-06-01 19:05:58,Clinton St & Newark St,HB409,Pershing Field,JC024,40.737430,-74.035710,40.742677,-74.051789,member
4,A2A50E81031E9E7F,electric_bike,2022-06-14 22:45:53,2022-06-14 22:49:39,Oakland Ave,JC022,Pershing Field,JC024,40.737604,-74.052478,40.742677,-74.051789,member


In [4]:
trip_df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [5]:
# Finding the count of each column to identify missing data
trip_df.count()

ride_id               415432
rideable_type         415432
started_at            415432
ended_at              415432
start_station_name    415370
start_station_id      415370
end_station_name      413978
end_station_id        413978
start_lat             415432
start_lng             415432
end_lat               414607
end_lng               414607
member_casual         415432
dtype: int64

#### From the above we can see that there are missing start station name, id, lat, lng and end station name, id, lat and lng. 

In [6]:
#unique start station ids, latitudes, longitudes
print(trip_df['start_station_id'].nunique())
print(trip_df['start_lat'].nunique())
print(trip_df['start_lng'].nunique())

#unique end station ids latitudes and longitudes
print(trip_df['end_station_id'].nunique())
print(trip_df['end_lat'].nunique())
print(trip_df['end_lng'].nunique())

167
53658
54625
320
407
407


#### The huge difference in the above numbers show same station id has variable latitudes and longitudes in the dataset. Also all start end station ids may or may not not have latitudes and longitudes. 

### Cleaning Latitude and Longitude data in the dataframe

In [7]:
#Creating new dataframe to clean up lat-long data
#Start coordinates df
start_cd = trip_df[['start_station_id','start_lat', 'start_lng']]
#Keeping only one value of lat and long per station id
start_cood = start_cd.groupby(['start_station_id']).head(1)
start_cood.head()

,start_station_id,start_lat,start_lng
0,HB409,40.737430,-74.035710
4,JC022,40.737604,-74.052478
7,HB601,40.742659,-74.032233
9,JC103,40.733670,-74.062500
12,JC072,40.712419,-74.038526


In [8]:
#End coordinates df
end_cd = trip_df[['end_station_id','end_lat', 'end_lng']]
#Keeping only one value of lat and long per station id
end_cood = end_cd.groupby(['end_station_id']).head(1)
end_cood.head()

,end_station_id,end_lat,end_lng
0,HB506,40.754090,-74.031600
3,JC024,40.742677,-74.051789
5,HB505,40.751867,-74.030377
6,HB101,40.735938,-74.030305
7,JC059,40.748716,-74.040443


In [9]:
#Adding back lat and lng to main trip df with inner join in order to remove any missing station ids and coordinates
trip_df = pd.merge(trip_df, start_cood, on='start_station_id')
trip_df = pd.merge(trip_df, end_cood, on='end_station_id')
trip_df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat_x,start_lng_x,end_lat_x,end_lng_x,member_casual,start_lat_y,start_lng_y,end_lat_y,end_lng_y
413973,6FFCF05B04DE923A,classic_bike,2023-07-11 18:43:11,2023-07-11 18:51:24,W 41 St & 8 Ave,6602.03,W 63 St & Broadway,7052.01,40.757464,-74.036431,40.771639,-73.982614,member,40.757464,-74.036431,40.771639,-73.982614
413974,61CCD77C303B3BAB,classic_bike,2023-07-18 12:54:02,2023-07-18 13:00:03,E 6 St & Avenue B,5584.04,E 14 St & 1 Ave,5779.10,40.755400,-74.040184,40.731393,-73.982867,member,40.755400,-74.040184,40.731393,-73.982867
413975,95B4D6DBF5D46729,classic_bike,2023-07-11 18:53:25,2023-07-11 19:00:42,W 63 St & Broadway,7052.01,W 89 St & Columbus Ave,7432.09,40.761369,-74.036242,40.788221,-73.970416,member,40.761369,-74.036242,40.788221,-73.970416
413976,215F8912F00B115A,classic_bike,2023-07-31 11:07:58,2023-07-31 11:21:54,West St & Liberty St,5184.08,E 16 St & 5 Ave,6022.04,40.737676,-74.052545,40.737262,-73.992390,member,40.737676,-74.052545,40.737262,-73.992390
413977,D4F344085F6C838B,classic_bike,2023-07-17 13:21:04,2023-07-17 13:50:39,Water St & Fletcher St,5024.10,Stanton St & Norfolk St,5445.07,40.744273,-74.034365,40.720747,-73.986274,casual,40.744273,-74.034365,40.720747,-73.986274


In [10]:
# Cleaning column names and keeping only required columns
trip_df = trip_df.rename(columns={"rideable_type":"bike_type",
                                 "start_lat_y":"start_lat",
                                  "start_lng_y": "start_lng",
                                  "end_lat_y": "end_lat",
                                  "end_lng_y":"end_lng"})
# Drop unwanted columns
trip_df = trip_df.drop(['start_lat_x', 'start_lng_x', 'end_lat_x', 'end_lng_x'], axis=1)
trip_df.head()

,ride_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,start_lat,start_lng,end_lat,end_lng
0,EEA3CD3B084370F4,classic_bike,2022-06-30 19:17:16,2022-06-30 19:27:13,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316
1,BF568C9B3C5FAE94,classic_bike,2022-06-02 12:46:44,2022-06-02 15:08:33,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316
2,DF6B06D460481A7E,classic_bike,2022-06-05 14:17:34,2022-06-05 14:27:12,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316
3,784EAB8AB33BDB7E,electric_bike,2022-06-28 22:10:40,2022-06-28 22:17:47,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316
4,8D5A3DC09D38FDBA,electric_bike,2022-06-11 12:17:52,2022-06-11 12:27:18,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316


In [11]:
trip_df.count()

ride_id               413978
bike_type             413978
started_at            413978
ended_at              413978
start_station_name    413978
start_station_id      413978
end_station_name      413978
end_station_id        413978
member_casual         413978
start_lat             413978
start_lng             413978
end_lat               413978
end_lng               413978
dtype: int64

### We could have decided to keep data with missing information to analyze those as well but decided to drop them while merging since its only 0.2% of the total data. Keeping bike_type = 'docked_bike' in the dataset though.  

### Adding time of day to data set

In [12]:
# https://datascientyst.com/extract-day-night-morning-afternoon-evening-from-pandas-python-datetime/#:~:text=If%20we%20divide%20the%20day%20night%20cycle%20into,%27Morning%27%2C%204%3A%20%27Noon%27%2C%205%3A%20%27Evening%27%2C%206%3A%20%27Night%27%7D%20df%5B%27day_part%27%5D.replace%28mapping%29
def get_part_of_day(h):
    if type(h) == int:
        return (
            "morning"
            if 5 <= h <= 11
            else "afternoon"
            if 12 <= h <= 16
            else "evening"
            if 17 <= h <= 19
            else "night"
            if 20 <= h <= 23
            else "late_night"
        )
    else:
        'error'

trip_df['time_of_day'] = pd.to_datetime(trip_df["started_at"]).dt.hour.apply(get_part_of_day)
trip_df.head()

,ride_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,start_lat,start_lng,end_lat,end_lng,time_of_day
0,EEA3CD3B084370F4,classic_bike,2022-06-30 19:17:16,2022-06-30 19:27:13,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316,evening
1,BF568C9B3C5FAE94,classic_bike,2022-06-02 12:46:44,2022-06-02 15:08:33,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316,afternoon
2,DF6B06D460481A7E,classic_bike,2022-06-05 14:17:34,2022-06-05 14:27:12,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316,afternoon
3,784EAB8AB33BDB7E,electric_bike,2022-06-28 22:10:40,2022-06-28 22:17:47,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316,night
4,8D5A3DC09D38FDBA,electric_bike,2022-06-11 12:17:52,2022-06-11 12:27:18,Clinton St & Newark St,HB409,Grand St & 14 St,HB506,casual,40.73743,-74.03571,40.75409,-74.0316,afternoon


In [13]:
# Sort df by start date
trip_df = trip_df.sort_values(by=['started_at'])
trip_df.head()

,ride_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,start_lat,start_lng,end_lat,end_lng,time_of_day
356873,CE100B693B500F4E,classic_bike,2022-06-01 00:02:02,2022-06-01 00:11:03,Columbus Dr at Exchange Pl,JC106,Essex Light Rail,JC038,member,40.716870,-74.032810,40.712774,-74.036486,late_night
360134,3432D5EFCBB4DC02,classic_bike,2022-06-01 00:09:47,2022-06-01 00:30:18,Madison St & 1 St,HB402,11 St & Washington St,HB502,member,40.738790,-74.039300,40.749985,-74.027150,late_night
92610,D02AC75222087AC7,electric_bike,2022-06-01 00:09:57,2022-06-01 00:24:07,Mama Johnson Field - 4 St & Jackson St,HB404,Mama Johnson Field - 4 St & Jackson St,HB404,member,40.743140,-74.040041,40.743140,-74.040041,late_night
360143,A334A07786E710F8,classic_bike,2022-06-01 00:09:57,2022-06-01 00:30:05,Madison St & 1 St,HB402,11 St & Washington St,HB502,member,40.738790,-74.039300,40.749985,-74.027150,late_night
286520,475B531BEE682B32,electric_bike,2022-06-01 00:10:18,2022-06-01 00:18:37,Grove St PATH,JC005,Liberty Light Rail,JC052,member,40.719586,-74.043117,40.711242,-74.055701,late_night


In [14]:
# Remove data where start and end time are same or end time less than start time
trip_df = trip_df.loc[trip_df.started_at < trip_df.ended_at]
trip_df.head()

,ride_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,start_lat,start_lng,end_lat,end_lng,time_of_day
356873,CE100B693B500F4E,classic_bike,2022-06-01 00:02:02,2022-06-01 00:11:03,Columbus Dr at Exchange Pl,JC106,Essex Light Rail,JC038,member,40.716870,-74.032810,40.712774,-74.036486,late_night
360134,3432D5EFCBB4DC02,classic_bike,2022-06-01 00:09:47,2022-06-01 00:30:18,Madison St & 1 St,HB402,11 St & Washington St,HB502,member,40.738790,-74.039300,40.749985,-74.027150,late_night
92610,D02AC75222087AC7,electric_bike,2022-06-01 00:09:57,2022-06-01 00:24:07,Mama Johnson Field - 4 St & Jackson St,HB404,Mama Johnson Field - 4 St & Jackson St,HB404,member,40.743140,-74.040041,40.743140,-74.040041,late_night
360143,A334A07786E710F8,classic_bike,2022-06-01 00:09:57,2022-06-01 00:30:05,Madison St & 1 St,HB402,11 St & Washington St,HB502,member,40.738790,-74.039300,40.749985,-74.027150,late_night
286520,475B531BEE682B32,electric_bike,2022-06-01 00:10:18,2022-06-01 00:18:37,Grove St PATH,JC005,Liberty Light Rail,JC052,member,40.719586,-74.043117,40.711242,-74.055701,late_night


In [15]:
trip_df.count()

ride_id               413952
bike_type             413952
started_at            413952
ended_at              413952
start_station_name    413952
start_station_id      413952
end_station_name      413952
end_station_id        413952
member_casual         413952
start_lat             413952
start_lng             413952
end_lat               413952
end_lng               413952
time_of_day           413952
dtype: int64

In [16]:
# Downloading cleaned and prepared dataset as csv for visualization
trip_df.to_csv('JCcitibikestripdata06-07-2022-23.csv')